In [1]:
import time

# --- Import Gremlin-Python modules ---
import gremlin_python
import gremlin_python.driver.serializer as se
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.client import Client
from gremlin_python.driver.serializer import GraphSONSerializersV2d0
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import Cardinality
from gremlin_python.process.traversal import Column
from gremlin_python.process.traversal import Direction
from gremlin_python.process.traversal import Operator
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import Pop
from gremlin_python.process.traversal import Scope
from gremlin_python.process.traversal import Barrier
from gremlin_python.process.traversal import Bindings
from gremlin_python.process.traversal import WithOptions
# ---------------------------------------

In [2]:
# Function: check response message from gremlin server
# return: (flag, result_list).
#          if successed, flag == True and result_list holds the message/data if available.
#          if failed, flag == False and result_list holds the raised error message.
def CheckResult(result):
    if result is not None:
        i = 0
        while result.done.running():
            time.sleep(0.1)
            i +=1
#             print (i)
            if i>300:
                break

        a = result.all()
        s = str(a)
        ss= s.lower()
        if 'finished' in ss:
            if 'error' in ss:
                return False, [s]
            else:
                return True, a.result()
        else:
            return False, [s]
    else:
        return False, []

In [ ]:
# ****************************************************
#  Connect to gremlin server
# ****************************************************

In [3]:
# --- Create client to do low-level operations with gremlin server ---
client = Client("ws://192.168.1.52:8182/gremlin",    # URL of gremlin server (janusgraph application interface)
                "g",                                 # name of traversal object
                username="",                         # username to access Janusgraph
                password="")                         # password to access Janusgraph
print (client)

In [4]:
res = client.submit('Gremlin.version()')
f,x = CheckResult(res)
print (f)
print ('Version of gremlin server is ',x)

True
Version of gremlin server is  ['3.3.3']


In [ ]:
# ****************************************************
#  Create a new graph property with Janusgraph
# ****************************************************

In [5]:
# --- Create graph on the remote gremlin server: ---
# res = client.submit("graph = TinkerGraph.open()")
res = client.submit("graph = JanusGraphFactory.open('inmemory')")
CheckResult(res)     # Ignore the 'rasised GremlinServerError' message and go ahead

(False, ['<Future at 0x23bf45f5f98 state=finished raised GremlinServerError>'])

In [6]:
# --- launch an operation of loading graphml data file on remote server: ---
res = client.submit("graph.io(graphml()).readGraph('c:/datatools/~test/air-routes.graphml')")
CheckResult(res)

(True, [None])

In [ ]:
# ****************************************************
#  Check/Open exitsting graph properties in Janusgraph
# ****************************************************

In [7]:
# <Method-1 of checking existings>
res = client.submit('ConfiguredGraphFactory.getGraphNames()')
CheckResult(res)

(False, ['<Future at 0x23bf45f9b70 state=finished raised GremlinServerError>'])

In [8]:
# <Method-2 of checking existings>
res = client.submit('ConfiguredGraphFactory.graphNames')
CheckResult(res)

(False, ['<Future at 0x23bf4617e10 state=finished raised GremlinServerError>'])

In [9]:
# --- Open the selected existing graph property on the remote Janusgraph server: ---
res = client.submit('graph = ConfiguredGraphFactory.open("SomeGraph")')
CheckResult(res)

(False, ['<Future at 0x23bf46171d0 state=finished raised GremlinServerError>'])

In [ ]:
# **************************************************************
#  Test client.submit approach which simulate a gremlin console
# **************************************************************

In [10]:
# <Test-1> --- Check graph's features ---
res = client.submit("graph.features()")
f,r = CheckResult(res)
if f:
    print (r[0])

FEATURES
> GraphFeatures
>-- Transactions: true
>-- Computer: true
>-- Persistence: true
>-- ThreadedTransactions: true
>-- ConcurrentAccess: true
> VariableFeatures
>-- Variables: true
>-- MixedListValues: false
>-- UniformListValues: false
>-- MapValues: true
>-- SerializableValues: false
>-- IntegerValues: true
>-- LongValues: true
>-- ByteValues: true
>-- StringArrayValues: true
>-- ByteArrayValues: true
>-- BooleanValues: true
>-- DoubleValues: true
>-- DoubleArrayValues: true
>-- LongArrayValues: true
>-- StringValues: true
>-- BooleanArrayValues: true
>-- FloatValues: true
>-- FloatArrayValues: true
>-- IntegerArrayValues: true
> VertexFeatures
>-- MetaProperties: true
>-- AddVertices: true
>-- RemoveVertices: true
>-- MultiProperties: true
>-- DuplicateMultiProperties: true
>-- UserSuppliedIds: false
>-- NumericIds: true
>-- StringIds: false
>-- AddProperty: true
>-- RemoveProperty: true
>-- CustomIds: false
>-- UuidIds: false
>-- AnyIds: false
> VertexPropertyFeatures
>-- User

In [11]:
# <Test-2> --- Create graph traversal object g ---
res = client.submit("g = graph.traversal()")
CheckResult(res)    # Ignore the 'rasised GremlinServerError' message and go ahead

(False, ['<Future at 0x23bf4629630 state=finished raised GremlinServerError>'])

In [12]:
# <Test-3> --- Optional sample code using client method ---
#
# WARNING: 
#    The following codes only work with the 'air-routes.graphml' data file!
#
r1 = client.submit("g.V().has('city','Sydney')").all().result()
print (type(r1))
print (r1)
print (len(r1),'vertices')
print (r1[0].id, r1[0].label)

<class 'list'>
[v[1343504], v[6418448], v[10350608], v[10911760], v[4247576], v[5713944], v[11829272], v[2875424], v[5152800], v[9494560], v[487464], v[14458920], v[3039296], v[12439616], v[1536088], v[9113688], v[13619288], v[5681248], v[8495200], v[1880256], v[7413952], v[7852224], v[12996800], v[45256], v[3412168], v[7323848], v[14049480], v[4264152], v[10354904], v[11784408]]
30 vertices
1343504 airport


In [13]:
r2 = client.submit("g.V().has('city','Sydney').has('country','AU')").all().result()
print (len(r2))
print (r2)

20
[v[1343504], v[10350608], v[4247576], v[5713944], v[11829272], v[2875424], v[14458920], v[3039296], v[1536088], v[9113688], v[13619288], v[5681248], v[8495200], v[7413952], v[12996800], v[45256], v[7323848], v[4264152], v[10354904], v[11784408]]


In [14]:
r3 = client.submit("g.V().range(1,10)").all().result()
print (len(r3))
print (r3)

9
[v[8208], v[12304], v[16400], v[20496], v[24592], v[28688], v[32784], v[36880], v[40976]]


In [15]:
r4 = client.submit("g.V().range(100,120)").all().result()
print (len(r4))
print (r4)

20
[v[413712], v[417808], v[421904], v[426000], v[430096], v[434192], v[438288], v[442384], v[446480], v[450576], v[454672], v[458768], v[462864], v[466960], v[471056], v[475152], v[479248], v[483344], v[487440], v[491536]]


In [16]:
# Close the client when having everything done.
client.close()

In [ ]:
# **************************************************************
#  Test python as a gremlin language variant
# **************************************************************

In [17]:
# Create graph and remote graph traversal objects (g) by RemoteConnection method:
# Take the most advantages out of Gremlin-Python

# statics.load_statics(globals())

graph = Graph()
print ('type of graph: ',type(graph))

# Binding local traversal object g with the remote traversal object g:
g = graph.traversal().withRemote(
        DriverRemoteConnection('ws://192.168.1.52:8182/gremlin', # URL of gremlin server (janusgraph application interface)
                               'g',                              # name of traversal object
                               username="",                      # username to access Janusgraph
                               password="")                      # password to access Janusgraph 
        )
print ('type of g: ',type(g))

type of graph:  <class 'gremlin_python.structure.graph.Graph'>
type of g:  <class 'gremlin_python.process.graph_traversal.GraphTraversalSource'>


In [18]:
x1 = g.V().has("city",'Sydney').toList()
print (len(x1))
print (x1)

30
[v[1343504], v[6418448], v[10350608], v[10911760], v[4247576], v[5713944], v[11829272], v[2875424], v[5152800], v[9494560], v[487464], v[14458920], v[3039296], v[12439616], v[1536088], v[9113688], v[13619288], v[5681248], v[8495200], v[1880256], v[7413952], v[7852224], v[12996800], v[45256], v[3412168], v[7323848], v[14049480], v[4264152], v[10354904], v[11784408]]


In [19]:
x2 = g.V().both()[1:3].toList()
print (len(x2))
print (x2)

2
[v[4160], v[4184]]


In [20]:
x3 = g.V().range(1,10).toList()
print (len(x3))
print (x3)

9
[v[8208], v[12304], v[16400], v[20496], v[24592], v[28688], v[32784], v[36880], v[40976]]


In [21]:
x4 = g.V(x3).toList()
print (len(x4))
print (x4)

9
[v[8208], v[12304], v[16400], v[20496], v[24592], v[28688], v[32784], v[36880], v[40976]]


In [22]:
x5 = g.V(x3).id().toList()
print (len(x5))
print (x5)

9
[8208, 12304, 16400, 20496, 24592, 28688, 32784, 36880, 40976]
